In [18]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils.np_utils import to_categorical
from nltk.corpus import stopwords
from nltk import word_tokenize
import re
import warnings
warnings.filterwarnings('ignore')

In [4]:
df=pd.read_csv("/content/IMDB Dataset.csv")

In [5]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.stem import PorterStemmer,WordNetLemmatizer
nltk.download( "wordnet")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [8]:
def clean_text(text):
  token=word_tokenize(text.lower()) #case conversion + tokenization.
  #non alpha removal.
  ftoken=[i for i in token if i.isalpha()]
  #stop words removal
  stpwd=stopwords.words("english")
  stoken=[i for i in ftoken if i not in stpwd]
  #lemma.
  lemma=WordNetLemmatizer ()
  ltoken=[lemma.lemmatize(i) for i in stoken]
  #joining list of msgs
  return " ".join(ltoken) 

In [9]:
df[ "review" ]=df["review"].astype(str) 

In [10]:
import nltk
nltk.download('omw-1.4') 

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [11]:
df["clean_review" ]=df["review"].apply(clean_text) 

In [12]:
df.head()

,review,sentiment,clean_review
0,One of the other reviewers has mentioned that ...,positive,one reviewer mentioned watching oz episode hoo...
1,A wonderful little production. <br /><br />The...,positive,wonderful little production br br filming tech...
2,I thought this was a wonderful way to spend ti...,positive,thought wonderful way spend time hot summer we...
3,Basically there's a family where a little boy ...,negative,basically family little boy jake think zombie ...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter mattei love time money visually stunnin...


In [13]:
x=df["clean_review" ]
x 

0        one reviewer mentioned watching oz episode hoo...
1        wonderful little production br br filming tech...
2        thought wonderful way spend time hot summer we...
3        basically family little boy jake think zombie ...
4        petter mattei love time money visually stunnin...
                               ...                        
49995    thought movie right good job creative original...
49996    bad plot bad dialogue bad acting idiotic direc...
49997    catholic taught parochial elementary school nu...
49998    going disagree previous comment side maltin on...
49999    one expects star trek movie high art fan expec...
Name: clean_review, Length: 50000, dtype: object

In [14]:
y=df["sentiment"]
y 

0        positive
1        positive
2        positive
3        negative
4        positive
           ...   
49995    positive
49996    negative
49997    negative
49998    negative
49999    negative
Name: sentiment, Length: 50000, dtype: object

In [21]:
MAX_NB_WORDS = 50000
MAX_SEQUENCE_LENGTH = 250
EMBEDDING_DIM = 100

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(df['review'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

X = tokenizer.texts_to_sequences(df['review'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Found 124266 unique tokens.
Shape of data tensor: (50000, 250)


In [27]:
Y = pd.get_dummies(df['sentiment']).values
print('Shape of label tensor:', Y.shape)




Shape of label tensor: (50000, 2)


In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(40000, 250) (40000, 2)
(10000, 250) (10000, 2)


In [29]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())




Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 250, 100)          5000000   
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 250, 100)         0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense_1 (Dense)             (None, 2)                 202       
                                                                 
Total params: 5,080,602
Trainable params: 5,080,602
Non-trainable params: 0
_________________________________________________________________
None


In [31]:
history = model.fit(X_train, Y_train, epochs=5, batch_size=64, validation_split=0.1)

Epoch 1/5
563/563 [==============================] - 289s 514ms/step - loss: 0.3944 - accuracy: 0.8183 - val_loss: 0.3957 - val_accuracy: 0.8290
Epoch 2/5
563/563 [==============================] - 290s 515ms/step - loss: 0.2358 - accuracy: 0.9054 - val_loss: 0.2970 - val_accuracy: 0.8825
Epoch 3/5
563/563 [==============================] - 290s 515ms/step - loss: 0.1400 - accuracy: 0.9497 - val_loss: 0.3270 - val_accuracy: 0.8813
Epoch 4/5
563/563 [==============================] - 289s 513ms/step - loss: 0.0931 - accuracy: 0.9680 - val_loss: 0.3707 - val_accuracy: 0.8817
Epoch 5/5
563/563 [==============================] - 289s 514ms/step - loss: 0.0653 - accuracy: 0.9784 - val_loss: 0.4171 - val_accuracy: 0.8808


In [32]:
accr = model.evaluate(X_test,Y_test)


313/313 [==============================] - 16s 51ms/step - loss: 0.4057 - accuracy: 0.8845


In [33]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.406
  Accuracy: 0.885
